In [1]:
from datasets import load_dataset, Dataset
import openai
import csv
dataset1 = load_dataset('json', data_files='sub_datasets/sub_dataset_1.json', split=None)
dataset2 = load_dataset('json', data_files='sub_datasets/sub_dataset_2.json', split=None)
dataset3 = load_dataset('json', data_files='sub_datasets/sub_dataset_3.json', split=None)
dataset4 = load_dataset('json', data_files='sub_datasets/sub_dataset_4.json', split=None)
dataset5 = load_dataset('json', data_files='sub_datasets/sub_dataset_5.json', split=None)

In [12]:
# 创建 OpenAI 客户端实例
# GPT-3.5-turbo
client = openai.OpenAI(api_key="sk-W9PtE5a3S9yYzcm581A77363176a40Cb80415bDa94544027", base_url="https://api.xiaoai.plus/v1")

def get_response(question, max_retries=5, wait_time=5):
    conversation_history = [{"role":"system","content":"you are a talented mathematician"}]
    for q in  question:
        conversation_history.append({"role": "user", "content":q})

        # 从模型中流式获取响应
        stream = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=conversation_history,
            stream=True,
        )
        response = ""

        for chunk in stream:
            try:
                if chunk.choices[0].delta.content is not None:
                    response_chunk = chunk.choices[0].delta.content
                    response += response_chunk
            except IndexError as e:
                pass
        conversation_history.append({"role": "assistant", "content": response})
    
    return response
    
GPT35res = []

print('在测试集上检查GPT-3.5-turbo表现')
for i in list(range(34, len(dataset5['train']))):
    print('%'*30, i, '%'*30)
    item = dataset5['train'][i]
    print('问题', item['text'])
    print('正确答案', item['answer_expressions'])
    
    question = [item['text'],
                '请输出你的最终答案并保证这个答案是最简形式，不需要任何其他内容，只输出答案本身']
    answer = get_response(question)
    print('gpt答案', answer)
    GPT35res.append([item['text'], item['answer_expressions'], answer])
    

在测试集上检查GPT-3.5-turbo表现
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 34 %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
问题 直线$y=2 x+3$与曲线$y=x^{2}$相交于$A$、$B$两点，则$|A B|$=?
正确答案 4*sqrt(5)
gpt答案 $|AB|=4\sqrt{5}$
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 35 %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
问题 若双曲线经过点$(6, \sqrt{3})$，且其渐近线方程为$y=\pm \frac{1}{3} x$，则此双曲线的标准方程?
正确答案 x^2/9-y^2=1
gpt答案 $\frac{x^2}{9} - y^2 = 1$
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 36 %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
问题 若直线$m x-y-2 m=0$与曲线$x=\sqrt{1+y^{2}}$有两个公共点，则实数$m$的取值范围为?
正确答案 (-oo,-1)+(1,+oo)
gpt答案 $m \in (-\infty, -2) \cup (0, 2)$
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 37 %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
问题 已知点$P(2, a)$在抛物线$C$: $y^{2}=4 x$上，则点$P$到抛物线$C$的焦点的距离为?
正确答案 3
gpt答案 |a|
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 38 %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
问题 已知方程$\frac{x^{2}}{-2-\lambda}+\frac{y^{2}}{1+2 \lambda}=-3$表示焦点在$y$轴上的椭圆，则$\lambda$的取值范围为?
正确答案 (-2, -1)
gpt答案 -4 \leq \lambda \leq -1
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% 39 %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
问题 已知动圆$M$与直线$y=2$相切，且与定圆$C$: $x^{2}+(y+3)^{2}

In [13]:
# 打开一个CSV文件进行写入
with open('GPT35_result51.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # 循环写入数据
    for row in GPT35res:
        writer.writerow(row)

print('数据已成功写入CSV文件.')

数据已成功写入CSV文件.
